# Cerebus llama3.1 70B PricePredictor

## Prompt:

Create a class that Trains a model given stock prices and indicators (rsi, on balance volume, detrended price oscilator), and produces target predictions of for the next 3 days open, low, and close. The model should also contain methods that will perform Baysian optimization of the LSTM's model hyper parameters. The class should use yfinance to pull daily stock price data. The class should also use mplfinance to plot the price data along with the rescaled open, low and close, predictions.

### Correction...

Please re-write the code using keras 3.5 which nolonger has keras.wrappers, and make use of scikeras.wrappers instead.


In [3]:
%matplotlib notebook

import yfinance as yf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import mplfinance as mpf
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from scipy import signal
from scipy.signal import find_peaks
from scipy.stats import kurtosis, skew
from sklearn.model_selection import RandomizedSearchCV
from scikeras.wrappers import KerasRegressor
from scipy.stats import randint
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV
from bayes_opt import BayesianOptimization

class StockPredictor:
    def __init__(self, ticker, start_date, end_date):
        self.ticker = ticker
        self.start_date = start_date
        self.end_date = end_date
        self.data = self.get_stock_data()
        self.orig_data = self.data.copy()
        self.scaler = None

    def get_stock_data(self):
        data = yf.download(self.ticker, start=self.start_date, end=self.end_date)
        # Move date from index to column
        data.reset_index(inplace=True)
        data.rename(columns={'index': 'Date'}, inplace=True)
        return data

    def calculate_indicators(self):
        data = self.data.copy()
        data['RSI'] = self.calculate_rsi(data['Close'])
        data['OBV'] = self.calculate_obv(data['Close'], data['Volume'])
        data['DPO'] = self.calculate_dpo(data['Close'])
        return data

    def calculate_rsi(self, prices, n=14):
        delta = prices.diff(1)
        up, down = delta.copy(), delta.copy()
        up[up < 0] = 0
        down[down > 0] = 0
        roll_up = up.rolling(n).mean()
        roll_down = down.rolling(n).mean().abs()
        RS = roll_up / roll_down
        RSI = 100.0 - (100.0 / (1.0 + RS))
        return RSI

    def calculate_obv(self, prices, volume):
        obv = volume.copy()
        obv[prices < obv.shift(1)] = -obv[prices < obv.shift(1)]
        obv[prices > obv.shift(1)] = obv[prices > obv.shift(1)]
        return obv.cumsum()

    def calculate_dpo(self, prices):
        dpo = prices.copy()
        dpo = dpo.rolling(30).mean()
        dpo = dpo.rolling(10).mean()
        return dpo

    def prepare_data(self):
        data = self.calculate_indicators()
        data['Date'] = data.index
        data['Open'] = data['Open'].shift(-3)
        data['Low'] = data['Low'].shift(-3)
        data['Close'] = data['Close'].shift(-3)
        data = data.dropna()
        data = data[['Date', 'Open', 'Low', 'Close', 'RSI', 'OBV', 'DPO']]
        return data

    def scale_data(self, data):
        self.scaler = MinMaxScaler()
        data[['Open', 'Low', 'Close', 'RSI', 'OBV', 'DPO']] = self.scaler.fit_transform(data[['Open', 'Low', 'Close', 'RSI', 'OBV', 'DPO']])
        return data

    # def create_model(self, units, dropout, learning_rate):
    #     model = Sequential()
    #     model.add(LSTM(units=units, return_sequences=True, input_shape=(self.data.shape[1], 1)))
    #     model.add(Dense(3))
    #     model.compile(loss='mean_squared_error', optimizer=Adam(learning_rate=learning_rate))
    #     return model

    def create_model(self, units, dropout, learning_rate):
        model = Sequential()
        model.add(LSTM(units=units, return_sequences=False, input_shape=(self.data.shape[1], 1)))
        model.add(Dense(3))
        model.compile(loss='mean_squared_error', optimizer=Adam(learning_rate=learning_rate))
        return model

    def train_model(self, data, units, dropout, learning_rate):
        X = data.drop(['Date', 'Open', 'Low', 'Close'], axis=1)
        y = data[['Open', 'Low', 'Close']]
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
        model = self.create_model(units, dropout, learning_rate)
        early_stopping = EarlyStopping(monitor='val_loss', patience=5, min_delta=0.001)
        model.fit(X_train, y_train, epochs=50, batch_size=32, validation_data=(X_test, y_test), callbacks=[early_stopping])
        return model

    def make_predictions(self, model, data):
        predictions = model.predict(data.drop(['Date', 'Open', 'Low', 'Close'], axis=1))
        return predictions

    # def plot_results(self, data, predictions):
    #     fig, ax = mpf.figure(figsize=(12, 6), style='yahoo', )
    #     ax.plot(data['Date'], data['Close'], label='Actual Close')
    #     ax.plot(data['Date'], predictions[:, 2], label='Predicted Close')
    #     ax.plot(data['Date'], data['Open'], label='Actual Open')
    #     ax.plot(data['Date'], predictions[:, 0], label='Predicted Open')
    #     ax.plot(data['Date'], data['Low'], label='Actual Low')
    #     ax.plot(data['Date'], predictions[:, 1], label='Predicted Low')
    #     ax.legend()
    #     mpf.show()

    def plot_results(self, data, predictions):
        # Copy st.orig_data dataframe columns open high low close and volume to data
        data = self.orig_data.copy()
        # Rescale predictions
        predictions = self.scaler.inverse_transform(predictions)
        
        addplot = [mpf.make_addplot(data['Close'], color='b', linestyle='-', title='Actual Close'),
                   mpf.make_addplot(predictions[:, 2], color='b', linestyle='--', title='Predicted Close'),
                   mpf.make_addplot(data['Open'], color='g', linestyle='-', title='Actual Open'),
                   mpf.make_addplot(predictions[:, 0], color='g', linestyle='--', title='Predicted Open'),
                   mpf.make_addplot(data['Low'], color='r', linestyle='-', title='Actual Low'),
                   mpf.make_addplot(predictions[:, 1], color='r', linestyle='--', title='Predicted Low')]
        mpf.plot(data, type='line', addplot=addplot, volume=False, style='yahoo')
        mpf.show()

    def bayesian_optimization(self):
        def optimize_model(units, dropout, learning_rate):
            model = self.create_model(int(units), dropout, learning_rate)
            X = self.data.drop(['Date', 'Open', 'Low', 'Close'], axis=1)
            y = self.data[['Open', 'Low', 'Close']]
            X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
            early_stopping = EarlyStopping(monitor='val_loss', patience=5, min_delta=0.001)
            model.fit(X_train, y_train, epochs=50, batch_size=32, validation_data=(X_test, y_test), callbacks=[early_stopping])
            loss = model.evaluate(X_test, y_test)
            return -loss

        optimizer = BayesianOptimization(f=optimize_model, pbounds={'units': (10, 100), 'dropout': (0.1, 0.5), 'learning_rate': (0.001, 0.1)}, random_state=42)
        optimizer.maximize(init_points=10, n_iter=20)
        print(optimizer.max)

    def run(self):
        data = self.prepare_data()
        data = self.scale_data(data)
        self.bayesian_optimization()
        model = self.train_model(data, units=50, dropout=0.2, learning_rate=0.001)
        predictions = self.make_predictions(model, data)
        self.plot_results(data, predictions)

predictor = StockPredictor('AAPL', '2020-01-01', '2022-02-26')
predictor.run()


[*********************100%***********************]  1 of 1 completed

|   iter    |  target   |  dropout  | learni... |   units   |
-------------------------------------------------------------
Epoch 1/50



/home/dsidlo/anaconda3/envs/pricepredict/lib/python3.12/site-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


14/14 ━━━━━━━━━━━━━━━━━━━━ 1s 16ms/step - loss: 14201.7969 - val_loss: 7662.2866
Epoch 2/50
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 6429.2754 - val_loss: 3140.2651
Epoch 3/50
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 2705.8406 - val_loss: 1478.1730
Epoch 4/50
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 1183.1271 - val_loss: 1225.3811
Epoch 5/50
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 908.8281 - val_loss: 1260.1660
Epoch 6/50
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 885.1492 - val_loss: 1266.9014
Epoch 7/50
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 903.6364 - val_loss: 1249.9878
Epoch 8/50
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 807.8162 - val_loss: 1240.0857
Epoch 9/50
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 915.3038 - val_loss: 1237.6212
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 1278.9469 
| 1         | -1.238e+0 | 0.2498    | 0.09512   | 75.88     |
Epoch 1/50


/home/dsidlo/anaconda3/envs/pricepredict/lib/python3.12/site-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


...

/home/dsidlo/anaconda3/envs/pricepredict/lib/python3.12/site-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


14/14 ━━━━━━━━━━━━━━━━━━━━ 3s 26ms/step - loss: 13976.1289 - val_loss: 9298.8232
Epoch 2/50
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 8577.8740 - val_loss: 5032.6216
Epoch 3/50
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 3986.3665 - val_loss: 2670.3948
Epoch 4/50
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 2394.9119 - val_loss: 1605.9760
Epoch 5/50
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 1319.2446 - val_loss: 1281.9768
Epoch 6/50
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 903.2936 - val_loss: 1225.2593
Epoch 7/50
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 973.0213 - val_loss: 1236.4270
Epoch 8/50
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 948.5600 - val_loss: 1242.2108
Epoch 9/50
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 914.2472 - val_loss: 1242.3135
Epoch 10/50
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 979.5397 - val_loss: 1243.9507
Epoch 11/50
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 915.0170 - val_loss: 1243.9497
4/4 ━━━━━━━━━━━━━━━━━━━

/home/dsidlo/anaconda3/envs/pricepredict/lib/python3.12/site-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


13/13 ━━━━━━━━━━━━━━━━━━━━ 2s 24ms/step - loss: 0.3524 - val_loss: 0.2744
Epoch 2/50
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.2629 - val_loss: 0.1921
Epoch 3/50
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.1833 - val_loss: 0.1205
Epoch 4/50
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.1068 - val_loss: 0.0820
Epoch 5/50
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.0654 - val_loss: 0.0800
Epoch 6/50
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.0704 - val_loss: 0.0749
Epoch 7/50
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.0576 - val_loss: 0.0723
Epoch 8/50
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.0580 - val_loss: 0.0702
Epoch 9/50
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.0606 - val_loss: 0.0684
Epoch 10/50
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.0550 - val_loss: 0.0678
Epoch 11/50
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.0548 - val_loss: 0.0647
Epoch 12/50
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.0569 - val_loss: 0.0633

ValueError: operands could not be broadcast together with shapes (502,3) (6,) (502,3) 